## Développement du "Modèle sur mesure avancé" avec des essais sur au moins deux word embeddings différents et en gardant celui qui permet d’obtenir les meilleures performances. Nous utiliserons Glove et FastText.
-  Nous allons construire et entraîner un réseau de neurones pour chaque embedding, évaluer les performances et enregistrer les expérimentations avec MLFlow. 
-  C'est la colonne clean_text_embeddings qui sera utilisée pour les embeddings.

### Charger le fichier CSV nettoyé

In [1]:
# Verification version de tensorflow
import tensorflow as tf
print(tf.__version__)

2.17.0


In [2]:
import os
import pandas as pd

# Chemin relatif pour charger les données nettoyées
file_path = os.path.join("..", "data", "cleaned_data_with_text_for_models.csv")

# Chargement du DataFrame nettoyé
data = pd.read_csv(file_path, index_col=0)

# Vérification du contenu du DataFrame

In [3]:
# Réinitialiser l'index pour récupérer toutes les colonnes, y compris 'text' si elle est utilisée comme index
data = data.reset_index()

# Vérifier les colonnes disponibles après réinitialisation de l'index
print(data.columns)


Index(['text', 'clean_text_tfidf', 'clean_text_embeddings', 'clean_text_bert',
       'target'],
      dtype='object')


In [4]:
# Dataframe info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 5 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   text                   1600000 non-null  object
 1   clean_text_tfidf       1592857 non-null  object
 2   clean_text_embeddings  1596608 non-null  object
 3   clean_text_bert        1600000 non-null  object
 4   target                 1600000 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 61.0+ MB


In [5]:
# Valeurs manquantes
print(data.isnull().sum())

text                        0
clean_text_tfidf         7143
clean_text_embeddings    3392
clean_text_bert             0
target                      0
dtype: int64


In [6]:
# Filtrer les lignes avec des valeurs manquantes dans la colonne 'clean_text_embeddings'
missing_embeddings = data[data['clean_text_embeddings'].isnull()]

# Afficher deux exemples pour analyser les raisons des valeurs manquantes
print(missing_embeddings[['text', 'clean_text_tfidf', 'clean_text_embeddings']].head(2))

             text clean_text_tfidf clean_text_embeddings
208    @mandayyy               NaN                   NaN
249  @mandayyy                 NaN                   NaN


In [7]:
# Supprimer les lignes avec des valeurs manquantes dans la colonne 'clean_text_embeddings'
data = data.dropna(subset=['clean_text_embeddings'])


In [8]:
# Vérifier les valeurs manquantes après suppression
print(data.isnull().sum())

text                        0
clean_text_tfidf         3751
clean_text_embeddings       0
clean_text_bert             0
target                      0
dtype: int64


### Étape 1 : Charger les embeddings GloVe et FastText
- Les liens pour télécharger les embeddings GloVe et FastText sont les suivants :
- https://nlp.stanford.edu/projects/glove/
- https://fasttext.cc/docs/en/english-vectors.html


In [9]:
import os
from gensim.models import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors

# Chemin relatif pour charger les fichiers d'embeddings
glove_path = os.path.join("..", "data", "glove.twitter.27B.100d.txt")
fasttext_path = os.path.join("..", "data", "crawl-300d-2M-subword.bin")

# Charger les embeddings GloVe
glove_model = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)

# Charger les embeddings FastText (format Facebook binaire)
fasttext_model = load_facebook_vectors(fasttext_path)


### Étape 2 : Créer une fonction pour générer une matrice d'embeddings
- Créer une matrice d'embeddings pour chaque modèle, basée sur le vocabulaire des données textuelles.

In [10]:
import numpy as np

# Fonction pour créer la matrice d'embeddings
def create_embedding_matrix(embedding_model, vocab, embedding_dim):
    embedding_matrix = np.zeros((len(vocab) + 1, embedding_dim))
    for word, i in vocab.items():
        if word in embedding_model:
            embedding_matrix[i] = embedding_model[word]
        else:
            # Si le mot n'est pas trouvé dans les embeddings, laisser un vecteur de zéros
            embedding_matrix[i] = np.zeros(embedding_dim)
    return embedding_matrix


### Étape 3 : Préparer le tokenizer et les données
- Avant de créer le réseau de neurones, nous devons tokeniser le texte et préparer les données.

In [11]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

# Tokenisation des textes nettoyés pour les word embeddings
num_words = 10000
max_sequence_length = 100

# Utiliser TextVectorization pour transformer les textes
tv_layer = tf.keras.layers.TextVectorization(
    max_tokens=num_words,
    output_mode='int',
    output_sequence_length=max_sequence_length
)

# Adapter TextVectorization sur les textes
tv_layer.adapt(data['clean_text_embeddings'])

# Convertir les textes en séquences
X = tv_layer(data['clean_text_embeddings'])

# Convertir le Tensor en tableau NumPy
X = X.numpy()

# Préparer les labels
y = data['target'].values  # Assurez-vous que y est également un tableau NumPy

# Division des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 4. Cellule countvectorizer pour les embeddings GloVe et FastText (transformer les textes en vecteurs)
- Nous utiliserons les textes déjà nettoyés pour les word embeddings à partir de la colonne clean_text_embeddings.

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
import numpy as np

# Sous-échantillonnage des données à 1 % pour tester rapidement le pipeline
data_sample = data.sample(frac=0.001, random_state=42)

# Utiliser CountVectorizer pour transformer les textes
num_words = 10000
vectorizer = CountVectorizer(max_features=num_words)

# Adapter et transformer les textes
X = vectorizer.fit_transform(data_sample['clean_text_embeddings'])

# Padding manuel des séquences pour une longueur maximale de 100
X_padded = hstack([X, np.zeros((X.shape[0], max(0, 100 - X.shape[1])))]).A

# Préparer les labels
y = data_sample['target'].values

# Division des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


### 5. Créer et entraîner les modèles avec GloVe et FastText
- Nous allons maintenant construire et entraîner les modèles avec les embeddings GloVe et FastText.

- a) Modèle de réseau de neurones :

In [13]:
import tensorflow as tf

# Utilisation de tf.keras pour construire le modèle
def build_model(embedding_matrix, input_length):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(input_dim=embedding_matrix.shape[0],
                                        output_dim=embedding_matrix.shape[1],
                                        weights=[embedding_matrix],
                                        input_length=input_length,
                                        trainable=False))  # Ne pas entraîner les embeddings
    model.add(tf.keras.layers.LSTM(128, return_sequences=False))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # Classification binaire
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


- b) Entraînement du modèle avec GloVe :

In [14]:
import numpy as np
import tensorflow as tf

# Créer la matrice d'embeddings pour GloVe
def create_embedding_matrix(glove_model, word_index, embedding_dim):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in glove_model:
            embedding_vector = glove_model[word]
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Créer un tokenizer et l'ajuster sur les textes originaux
# Remplacez 'data_sample' par le DataFrame qui contient vos données textuelles
num_words = 10000  # Assurez-vous que ce nombre correspond à vos besoins
texts = data_sample['clean_text_embeddings']
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(texts)

In [15]:
# Créer la matrice d'embeddings pour GloVe
embedding_dim_glove = 100  # Taille des vecteurs d'embeddings pour GloVe
embedding_matrix_glove = create_embedding_matrix(glove_model, tokenizer.word_index, embedding_dim_glove)

# Construire et entraîner le modèle avec GloVe
model_glove = build_model(embedding_matrix_glove, input_length=100)
history_glove = model_glove.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5


C:\Users\pat\.conda\envs\P7te\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


40/40 ━━━━━━━━━━━━━━━━━━━━ 108s 3s/step - accuracy: 0.5160 - loss: 0.6901 - val_accuracy: 0.5344 - val_loss: 0.6917
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step - accuracy: 0.5555 - loss: 0.6868 - val_accuracy: 0.5312 - val_loss: 0.6901
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step - accuracy: 0.5279 - loss: 0.6862 - val_accuracy: 0.5312 - val_loss: 0.6889
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 108s 3s/step - accuracy: 0.5561 - loss: 0.6838 - val_accuracy: 0.5219 - val_loss: 0.6907
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 105s 3s/step - accuracy: 0.5471 - loss: 0.6824 - val_accuracy: 0.5125 - val_loss: 0.6904


- c) Entraînement du modèle avec FastText :

In [16]:
# Créer la matrice d'embeddings pour FastText
embedding_dim_fasttext = 300  # Taille des vecteurs d'embeddings pour FastText
embedding_matrix_fasttext = create_embedding_matrix(fasttext_model, tokenizer.word_index, embedding_dim_fasttext)

# Construire et entraîner le modèle avec FastText
input_length = 100  # La longueur des séquences padding
model_fasttext = build_model(embedding_matrix_fasttext, input_length=input_length)

# Entraîner le modèle
history_fasttext = model_fasttext.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/5


C:\Users\pat\.conda\envs\P7te\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


40/40 ━━━━━━━━━━━━━━━━━━━━ 113s 3s/step - accuracy: 0.5256 - loss: 0.6908 - val_accuracy: 0.4719 - val_loss: 0.6948
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 102s 3s/step - accuracy: 0.4973 - loss: 0.6922 - val_accuracy: 0.5031 - val_loss: 0.6959
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 107s 3s/step - accuracy: 0.5529 - loss: 0.6782 - val_accuracy: 0.5031 - val_loss: 0.6921
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 108s 3s/step - accuracy: 0.5401 - loss: 0.6848 - val_accuracy: 0.5188 - val_loss: 0.7052
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 113s 3s/step - accuracy: 0.5280 - loss: 0.6919 - val_accuracy: 0.5281 - val_loss: 0.6930


### 6. Enregistrer les expérimentations avec MLFlow

- a) Enregistrer les résultats pour GloVe :

In [ ]:
from pathlib import Path
import mlflow
import mlflow.keras

# Chemin absolu pour le dossier "mlruns"
mlruns_path = Path("../mlruns").resolve()

# Configuration du chemin pour stocker les artefacts de MLFlow
mlflow.set_tracking_uri(mlruns_path.as_uri())

# Créer une nouvelle expérience ou utiliser une existante
experiment_name = "GloVe_Embedding_Experiment"
mlflow.set_experiment(experiment_name)

# Démarrer une nouvelle session MLFlow pour GloVe
with mlflow.start_run():
    mlflow.log_param("embedding", "GloVe")
    mlflow.log_param("embedding_dim", embedding_dim_glove)
    mlflow.log_metric("accuracy", model_glove.evaluate(X_test, y_test)[1])
    
    # Enregistrer le modèle GloVe comme artefact
    mlflow.keras.log_model(model_glove, "model_glove")

print(f"Modèle GloVe enregistré dans {mlruns_path}.")


In [17]:
# import mlflow
# import mlflow.keras
# import os
# 
# # Chemin relatif pour le dossier "models"
# relative_models_path = os.path.join("..", "models")
# 
# # Configuration du chemin pour stocker les artefacts de MLFlow
# mlflow.set_tracking_uri(relative_models_path)
# 
# # Créer une nouvelle expérience ou utiliser une existante
# experiment_name = "GloVe_Embedding_Experiment"
# mlflow.set_experiment(experiment_name)
# 
# # Démarrer une nouvelle session MLFlow pour GloVe
# with mlflow.start_run():
#     mlflow.log_param("embedding", "GloVe")
#     mlflow.log_param("embedding_dim", embedding_dim_glove)
#     mlflow.log_metric("accuracy", model_glove.evaluate(X_test, y_test)[1])
#     
#     # Enregistrer le modèle GloVe comme artefact
#     mlflow.keras.log_model(model_glove, "model_glove")
# 
# print(f"Modèle GloVe enregistré dans {relative_models_path}.")


2024/10/20 09:41:23 INFO mlflow.tracking.fluent: Experiment with name 'GloVe_Embedding_Experiment' does not exist. Creating a new experiment.
2024/10/20 09:41:24 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:

10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5026 - loss: 0.6920


2024/10/20 09:41:41 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/20 09:41:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle GloVe enregistré dans ..\models.


In [18]:
# import mlflow
# import mlflow.keras
# 
# # Enregistrer les résultats pour GloVe
# mlflow.start_run(run_name="GloVe Embedding")
# mlflow.log_param("embedding", "GloVe")
# mlflow.log_param("embedding_dim", embedding_dim_glove)
# mlflow.log_metric("accuracy", model_glove.evaluate(X_test, y_test)[1])
# mlflow.keras.log_model(model_glove, "model_glove")
# mlflow.end_run()

- b) Enregistrer les résultats pour FastText :

In [ ]:
from pathlib import Path
import mlflow
import mlflow.keras

# Chemin absolu pour le dossier "mlruns" en utilisant pathlib
mlruns_path = Path("../mlruns").resolve()

# Configuration du chemin pour stocker les artefacts de MLFlow
mlflow.set_tracking_uri(mlruns_path.as_uri())

# Créer une nouvelle expérience ou utiliser une existante
experiment_name = "FastText_Embedding_Experiment"
mlflow.set_experiment(experiment_name)

# Démarrer une nouvelle session MLFlow pour FastText
with mlflow.start_run():
    mlflow.log_param("embedding", "FastText")
    mlflow.log_param("embedding_dim", embedding_dim_fasttext)
    mlflow.log_metric("accuracy", model_fasttext.evaluate(X_test, y_test)[1])
    
    # Enregistrer le modèle FastText comme artefact
    mlflow.keras.log_model(model_fasttext, "model_fasttext")

print(f"Modèle FastText enregistré dans {mlruns_path}.")


In [19]:
# import mlflow
# import mlflow.keras
# import os
# 
# # Chemin relatif pour le dossier "models"
# relative_models_path = os.path.join("..", "models")
# 
# # Configuration du chemin pour stocker les artefacts de MLFlow
# mlflow.set_tracking_uri(relative_models_path)
# 
# # Créer une nouvelle expérience ou utiliser une existante
# experiment_name = "FastText_Embedding_Experiment"
# mlflow.set_experiment(experiment_name)
# 
# # Démarrer une nouvelle session MLFlow pour FastText
# with mlflow.start_run():
#     mlflow.log_param("embedding", "FastText")
#     mlflow.log_param("embedding_dim", embedding_dim_fasttext)
#     mlflow.log_metric("accuracy", model_fasttext.evaluate(X_test, y_test)[1])
#     
#     # Enregistrer le modèle FastText comme artefact
#     mlflow.keras.log_model(model_fasttext, "model_fasttext")

print(f"Modèle FastText enregistré dans {relative_models_path}.")


2024/10/20 09:41:58 INFO mlflow.tracking.fluent: Experiment with name 'FastText_Embedding_Experiment' does not exist. Creating a new experiment.


10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.5180 - loss: 0.6924


2024/10/20 09:42:11 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/20 09:42:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle FastText enregistré dans ..\models.


In [20]:
# # Enregistrer les résultats pour FastText
# mlflow.start_run(run_name="FastText Embedding")
# mlflow.log_param("embedding", "FastText")
# mlflow.log_param("embedding_dim", embedding_dim_fasttext)
# mlflow.log_metric("accuracy", model_fasttext.evaluate(X_test, y_test)[1])
# mlflow.keras.log_model(model_fasttext, "model_fasttext")
# mlflow.end_run()

### 7. Conclusion
- Ce code utilise le fichier cleaned_data_with_text_for_models.csv et applique les word embeddings GloVe et FastText. Nous avons modifié le code pour utiliser les textes déjà nettoyés dans la colonne clean_text_embeddings. Après avoir entraîné les modèles, nous enregistrons les résultats et les modèles dans MLFlow afin de comparer les performances des deux embeddings et de sélectionner le meilleur.